In [2]:
#SuperstoreSales
#import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px

#Load the file
df = pd.read_csv("/content/drive/MyDrive/GitHub/train.csv", index_col='Row ID')
# Convert dates and clean data
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d/%m/%Y')
df.sort_values(by=['Order Date'], ascending=True, inplace=True)
#Set order date as index
df.set_index('Order Date', inplace =True)
print(df.head())

                  Order ID  Ship Date       Ship Mode Customer ID  \
Order Date                                                          
2015-01-03  CA-2015-103800 2015-01-07  Standard Class    DP-13000   
2015-01-04  CA-2015-112326 2015-01-08  Standard Class    PO-19195   
2015-01-04  CA-2015-112326 2015-01-08  Standard Class    PO-19195   
2015-01-04  CA-2015-112326 2015-01-08  Standard Class    PO-19195   
2015-01-05  CA-2015-141817 2015-01-12  Standard Class    MB-18085   

            Customer Name      Segment        Country          City  \
Order Date                                                            
2015-01-03  Darren Powers     Consumer  United States       Houston   
2015-01-04  Phillina Ober  Home Office  United States    Naperville   
2015-01-04  Phillina Ober  Home Office  United States    Naperville   
2015-01-04  Phillina Ober  Home Office  United States    Naperville   
2015-01-05     Mick Brown     Consumer  United States  Philadelphia   

                  

In [3]:
#Checking for null values
df.isnull().sum()

,0
Order ID,0
Ship Date,0
Ship Mode,0
Customer ID,0
Customer Name,0
Segment,0
Country,0
City,0
State,0
Postal Code,11


In [4]:
df[df['Postal Code'].isnull()]
#City is Burlington for all the cases
df[df['City'] == 'Burlington']
#Add the postal code manually - 27217
df['Postal Code'].fillna(27217.0)

,Postal Code
Order Date,
2015-01-03,77095.0
2015-01-04,60540.0
2015-01-04,60540.0
2015-01-04,60540.0
2015-01-05,19143.0
...,...
2018-12-30,94533.0
2018-12-30,10009.0
2018-12-30,47201.0


In [5]:
#ExploratoryDataAnalysis
#Sales_by_category
import plotly.express as px
sales_by_category = df.groupby('Category')['Sales'].sum().reset_index()
fig1 = px.pie(sales_by_category, names='Category', values='Sales', title='Sales by Category', color='Category')
fig1.show()


There is no clear Category dominating the sales. Technology products contribute to 36.6% of the sales and is the highest by category. Furniture and office supplies are similar with 32.2% and 31.2% respectively.

In [ ]:
sales_w_city = df.groupby('City')['Sales'].sum().sort_values(ascending=True).reset_index()
print(sales_w_city)

              City       Sales
0          Abilene       1.392
1           Elyria       1.824
2          Jupiter       2.064
3        Pensacola       2.214
4     Ormond Beach       2.808
..             ...         ...
524   Philadelphia  108841.749
525  San Francisco  109041.120
526        Seattle  116106.322
527    Los Angeles  173420.181
528  New York City  252462.547

[529 rows x 2 columns]


In [ ]:
#Top 5 cities by sales
sales_city = df.groupby('City')['Sales'].sum().sort_values(ascending=False).reset_index()
top_5_sales_city = sales_city.head(5)
print(top_5_sales_city)





            City       Sales
0  New York City  252462.547
1    Los Angeles  173420.181
2        Seattle  116106.322
3  San Francisco  109041.120
4   Philadelphia  108841.749


In [ ]:

# Group by City and sum Sales
sales_city = df.groupby('City')['Sales'].sum().sort_values(ascending=False).reset_index()

# Select the top 5 cities
top_5_sales_city = sales_city.head(5).copy()

# Calculate total sales
total_sales = sales_city['Sales'].sum()

# Calculate percentage contribution of each top city
top_5_sales_city['Percentage'] = (top_5_sales_city['Sales'] / total_sales) * 100

# Create a bar chart
fig3 = px.bar(
    top_5_sales_city,
    x='City',
    y='Percentage',
    title='Top 5 Cities by Sales (% of Total Sales)',
    text=top_5_sales_city['Percentage'].apply(lambda x: f'{x:.2f}%'),  # Show whole number percentages
    labels={'Percentage': 'Percentage of Total Sales', 'City': 'City'},
    color='City'
)

fig3.show()


New York is considerably higher than all the other cities as it contributes to 11.16% of the total sales.

Distribution of Shipping duration

In [ ]:
df.index = pd.to_datetime(df.index)
df['Shipping Duration'] = (df['Ship Date'] - df.index).dt.days
print(df['Shipping Duration'])
#Counts the occurrences of each unique shipping duration.
shipping_duration_counts = df['Shipping Duration'].value_counts().sort_index()

Order Date
2015-01-03    4
2015-01-04    4
2015-01-04    4
2015-01-04    4
2015-01-05    7
             ..
2018-12-30    4
2018-12-30    4
2018-12-30    6
2018-12-30    4
2018-12-30    4
Name: Shipping Duration, Length: 9800, dtype: int64


In [ ]:
fig5 = px.bar(
    x=shipping_duration_counts.index,
    y=shipping_duration_counts.values,
    labels={'x': 'Shipping Duration (days)', 'y': 'Frequency'},
    title='Distribution of Shipping Duration',
    text=shipping_duration_counts.values,
)

fig5.update_traces(marker_color='teal', textposition='outside')
fig5.show()


Most no. of products take 4 or 5 days to ship.

Sales by Region

This provides a clear view of which regions perform best.

In [ ]:
sales_by_region = df.groupby('Region')['Sales'].sum().reset_index()
fig6 = px.pie(sales_by_region, names='Region', values='Sales', title='Sales by Region', color='Region')
fig6.show()

The West accounts for 31.4% of the total sales and is almost double that of South with only 17.2%. East also contributes to 29.6% while Central is only 21.8%.

Sales by Sub-Category

In [ ]:
sales_by_sub_cat = df.groupby('Sub-Category')['Sales'].sum().reset_index()
fig7 = px.pie(sales_by_sub_cat, names='Sub-Category', values='Sales', title='Sales by Sub-Category', color='Sub-Category')
fig7.show()
#

Phones and chairs are the prodcuts with the highest sales values, accounting for close to 29% of the total sales.

Top 10 Customers by Sales

In [ ]:
top_cust = df.groupby('Customer Name')['Sales'].sum().sort_values(ascending=False).reset_index()
top_10_cust = top_cust.head(10)
fig8 = px.bar(
    top_10_cust,
    x='Customer Name',
    y='Sales',
    title='Top 10 Customers by Sales',
  )

fig8.show()


Sean Miller is the top customer with around 25k in sales.

In [ ]:
#resample('Q').sum() groups sales data by quarter.
quarterly_sales = df['Sales'].resample('QE').sum().reset_index()


# Create visualization
#px.line() creates a line chart with markers to clearly show sales trends over quarters
fig = px.line(quarterly_sales, x='Order Date', y='Sales', title='Quarterly Sales Trend',
              labels={'Order Date': 'Quarter', 'Sales': 'Total Sales'}, markers=True)

# Show plot
fig.show()

It is clear that the first quarter sales are consistently low over the years. The sales pick up as the year goes on and increases progressively thtough the next 3 quarters and drops again in the first quarter of the year.

In [ ]:
fig_bar = px.bar(quarterly_sales, x='Order Date', y='Sales', title='Quarterly Sales Trend (Bar Chart)',
                 labels={'Order Date': 'Quarter', 'Sales': 'Total Sales'}, text_auto=True)
fig_bar.show()

In [ ]:
monthly_sales = df['Sales'].resample('M').sum().reset_index()

# Line chart for monthly sales
fig_line = px.line(monthly_sales, x='Order Date', y='Sales', title='Monthly Sales Trend (Line Chart)',
                   labels={'Order Date': 'Month', 'Sales': 'Total Sales'}, markers=True)
fig_line.show()

# Bar chart for monthly sales
fig_bar = px.bar(monthly_sales, x='Order Date', y='Sales', title='Monthly Sales Trend (Bar Chart)',
                 labels={'Order Date': 'Month', 'Sales': 'Total Sales'}, text_auto=True)
fig_bar.show()

<ipython-input-8-5be8fc7b1cf6>:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



February is the lowest performing consistently. January is also amongst the lowest performing months. The sales go up considerably in March to end the firs quarter on a good note.

September, November and Decemeber are consistently the highest performing months.

In [ ]:
df.reset_index(inplace=True)

#Extract the year
df['Year'] = df['Order Date'].dt.year

# Group by Year and Category, then sum sales
yearly_sales = df.groupby(['Year', 'Category'])['Sales'].sum().reset_index()

# Find the category with the highest sales for each year
top_category_per_year = yearly_sales.loc[yearly_sales.groupby('Year')['Sales'].idxmax()]

# Bar chart for highest sales category per year
fig = px.bar(top_category_per_year, x='Year', y='Sales', color='Category',
             title='Highest Sales by Category Each Year',
             labels={'Sales': 'Total Sales', 'Year': 'Year'},
             text_auto=True)

# Show plot
fig.show()

It is clear that Technology products are the most sold products in all but just one year (2106). The sales of technology products are increasing every year as well.

In [ ]:
yearly_sales_sub = df.groupby(['Year', 'Sub-Category'])['Sales'].sum().reset_index()

# Find the category with the highest sales for each year
top_sub_category_per_year = yearly_sales_sub.loc[yearly_sales_sub.groupby('Year')['Sales'].idxmax()]

# Bar chart for highest sales category per year
fig9 = px.bar(top_sub_category_per_year, x='Year', y='Sales', color='Sub-Category',
             title='Highest Sales by Sub-Category Each Year',
             labels={'Sales': 'Total Sales', 'Year': 'Year'},
             text_auto=True)

# Show plot
fig9.show()

Looking deeper into the sub-category of products sold, it can be seen that phones were the highest sold product sub-category in 2015 and 2018 while it was chairs in 2016 and 2017. There is a 35% increase in phone sales amount in 2018.

In [10]:
sales_segment = df.groupby(['Segment'])['Sales'].sum().reset_index()

# Find the category with the highest sales for each year
sales_segment['Sales_Percent'] = (sales_segment['Sales'] / sales_segment['Sales'].sum()) * 100

# Sort values by percentage (optional for better visualization)
sales_segment = sales_segment.sort_values(by='Sales_Percent', ascending=False)

# Bar chart showing percentage contribution
fig10 = px.bar(sales_segment, x='Segment', y='Sales_Percent', color='Segment',
               title='Sales Percentage by Segment',
               labels={'Sales_Percent': 'Percentage of Total Sales (%)', 'Segment': 'Segment'},
               text_auto='.2f')  # Format text to show two decimal places

# Show plot
fig10.show()


Consumer products are contributing to more than 50% of the total sales